In [1]:
import os,sys; sys.path.insert(0,os.path.abspath('../..'))
from borges import *
from chadwyck import *

In [2]:
class ChadwyckCorpus(BaseCorpus):
    id='chadwyck'
    name='Chadwyck'

    def compile(self):
        # setup
        if not self.url_metadata or not self.url_raw: 
            log.error(f'url_metadata or url_raw not set')
            return
        
        old=[]
        for fn,xml in self.iter_raw():
            fnmeta = get_meta_from_raw_xml(xml, fn)
            _id = f'_{self.id}/{os.path.splitext(fnmeta["fn_raw"])[0]}'
            ometa={
                KEY_ID:_id,
                **fnmeta
            }

            num_rows = 0
            num_words = 0
            tld=[]
            for tdx in iter_passages(xml):
                num_rows += 1
                num_words += len(tdx.get('text','').split())
                tdx={**{KEY_ID:os.path.join(_id, f'{tdx["tag_i"]:06}')}, **tdx}
                tld.append(tdx)
                yield {**ometa, **tdx}

            
            ometa['num_rows'] = num_rows
            ometa['num_words'] = num_words
            old.append(ometa)

            # for tdx in tld:
            #     yield {**ometa, **tdx}
            # ometa['_doc'] = tld

            # yield ometa
        

        df = pd.DataFrame(old).set_index(KEY_ID)
        df.to_csv(self.path_metadata)

            
        


    
            

        




C=ChadwyckCorpus()
f=C.compile()

In [3]:
import pysolr
solr = pysolr.Solr('http://localhost:8983/solr/phoeb/', always_commit=True)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":0,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"-24"}},\n  "status":"OK"}\n'

In [4]:
for x in C.compile():
    solr.add(x)

 81%|████████▏ | 8312/10208 [12:25<02:13, 14.21it/s]  

In [ ]:
x

{'id': '_chadwyck/Eighteenth-Century_Fiction/paltock.01/000006',
 'title': 'Peter Wilkins (1751)',
 'idref': 'Z200043982',
 'author': 'Paltock, Robert, 1697-1767',
 'year': '1751',
 'pub': 'Printed for J. Robinson, ... and R. Dodsley, [etc.]',
 'type': 'BOOK',
 'name': 'Vol. 1',
 'nation': 'British',
 'medium': 'Fiction',
 'subcorpus': 'Eighteenth-Century_Fiction',
 'fn_raw': 'Eighteenth-Century_Fiction/paltock.01.new',
 'tree': '1.body > 2.doc > 3.pb > 4.div2 > 5.comhd2 > 6.idref',
 'tag_i': 6,
 'tag': 'idref',
 'text': 'Z200043982'}

In [ ]:
# list(solr.search('text:Z200043982'))

In [ ]:
list(solr.search('text:bread'))

[{'id': '_chadwyck/Eighteenth-Century_Fiction/paltock.01/000670',
  'title': ['Peter Wilkins (1751)'],
  'idref': ['Z200043982'],
  'author': ['Paltock, Robert, 1697-1767'],
  'year': [1751],
  'pub': ['Printed for J. Robinson, ... and R. Dodsley, [etc.]'],
  'type': ['BOOK'],
  'name': ['Vol. 1'],
  'nation': ['British'],
  'medium': ['Fiction'],
  'subcorpus': ['Eighteenth-Century_Fiction'],
  'fn_raw': ['Eighteenth-Century_Fiction/paltock.01.new'],
  'tree': ['1.body > 2.doc > 3.pb > 4.div2 > 401.pb > 488.pb > 587.div3 > 617.pb > 667.p > 670.s'],
  'tag_i': 670,
  'tag': ['s'],
  'text': ['Why was I not placed out to some laborious\nCraft, where I might have drudged for Bread in\nmy proper Station! '],
  '_version_': 1756293272563613696},
 {'id': '_chadwyck/Eighteenth-Century_Fiction/paltock.01/000266',
  'title': ['Peter Wilkins (1751)'],
  'idref': ['Z200043982'],
  'author': ['Paltock, Robert, 1697-1767'],
  'year': [1751],
  'pub': ['Printed for J. Robinson, ... and R. Dodsley, 